In [ ]:
import time
from datetime import datetime, timedelta
from pprint import pprint
import dateutil.parser
import json
import re
import os
import portalocker
import logging
#from splunk_http_event_collector import http_event_collector
import timeout_decorator
import configparser
from __future__ import absolute_import
from __future__ import print_function
import splunklib.client as client
import splunklib.results as results
import sys
from time import sleep

#if len(sys.argv) < 2:
#    print "Pass the name of a config file as argument 1"
#    exit(1)

#if not os.path.exists(sys.argv[1]):
#    print "Cannot find the configuration file: "+sys.argv[1]
#    exit(1)
config_file='export1.conf'



In [ ]:
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s',
                    datefmt='%m-%d %H:%M:%S')

logger = logging.getLogger()
logger.handlers[0].stream = sys.stdout
logger.setLevel(logging.INFO) # DEBUG,INFO,WARNING,ERROR,CRITICAL

In [10]:
def load_config():
    global config
    config = configparser.ConfigParser()
    config.read(config_file)

def create_output_dir():
    path=config.get('export', 'directory')
    if not os.path.exists(path):
        os.makedirs(path)

def build_search_string(partition_in):
    logging.info('build_search_string-start')
    logging.debug('indexes: %s',config.get('search', 'indexes'))
    logging.debug('extra: %s',config.get('search', 'extra'))
    s='search index='+partition_in["index"]
    #s+=' earliest='+partition_in["earliest"]
    #s+=' latest='+partition_in["latest"]
    s+=' '+config.get('search', 'extra')
    
    logging.debug('s: %s',s)
    logging.info('build_search_string-end')
    return s

def explode_date_range(begin_date_in: str,end_date_in: str,interval_unit_in: str,interval_in: int):
    logging.info('explode_date_range-start')
    logging.debug('begin_date_in: %s',begin_date_in)
    logging.debug('end_date_in: %s',end_date_in)
    begin_date = dateutil.parser.parse(begin_date_in)
    end_date = dateutil.parser.parse(end_date_in)

    begin_current=begin_date
    result = []
    
    while begin_current < end_date:
        end_current = begin_current+timedelta(**{interval_unit_in: interval_in})-timedelta(seconds=1)
        result.append ([begin_current,end_current])
        begin_current=end_current+timedelta(seconds=1)
    logging.info('explode_date_range-end')
    return result

def write_search_partitions(date_array_in):
    logging.info('write_search_partitions-start')
    index_list=config.get('search', 'indexes').split(",")
    
    result_list = []
    i=1
    for index_entry in index_list:
        for entry in date_array_in:
            search_partition={'id':i,'index':index_entry,
                'earliest': entry[0].isoformat(),'latest':entry[1].isoformat(),
                #'earliest': entry[0].strftime("%m/%d/%Y:%H:%M:%S"),'latest':entry[1].strftime("%m/%d/%Y:%H:%M:%S"),
                             'status':'not started','pid':''}
            result_list.append(search_partition)
            i+=1
    
    search_partitions=json.dumps(result_list, indent=2, sort_keys=True)
    #print(search_partitions)      
    
    
    with open("partitions.info", "w") as outfile:
        outfile.write(search_partitions)
        
    logging.info('write_search_partitions-end')

def update_partition_status(partition_file_in,partition_in,status_in):
    with open(partition_file_in,'r+') as f:
        portalocker.lock(f, portalocker.LOCK_EX)
        data = json.load(f)
        search_partition_out=None
        for search_partition in data:
            if search_partition["id"]==partition_in["id"]:
                #pprint(search_partition["earliest"])
                search_partition["status"]=status_in
                #return search_partition
                break
            
        json_object = json.dumps(data, indent=4)
        f.seek(0)
        f.write(json_object)
        f.truncate()
    
def get_search_partition(partition_file_in):
    with open(partition_file_in,'r+') as f:
        portalocker.lock(f, portalocker.LOCK_EX)
        data = json.load(f)
        search_partition_out=None
        for search_partition in data:
            if search_partition["status"]=='not started':
                #pprint(search_partition["earliest"])
                search_partition["status"]='assigned'
                search_partition["pid"]=os.getpid()
                search_partition_out=search_partition
                #return search_partition
                break
                #pid = os.getpid()
        #with open("sample.json", "w") as outfile:
        #outfile.write(json_object)
        json_object = json.dumps(data, indent=4)
        f.seek(0)
        f.write(json_object)
        f.truncate()
        return search_partition_out
        #f.write(json_object)
        #pprint(data)

        


def search_export(service_in,search_in,partition_in):
    logging.info('search_export-start')
    

    logging.debug(service_in)
    kwargs_export = {"search_mode": "normal",
                     'earliest_time': partition_in["earliest"],
                     'latest_time': partition_in["latest"],
                     "output_mode": "json"}
    job = service_in.jobs.export(search_in, **kwargs_export)

    logging.info("search_export-end")
    return job
        
def dispatch_searches(partition_file_in):
    logging.info('dispatch_searches-start')

    while True:
        partition_out=get_search_partition('partitions.info')

        if partition_out is not None:
            search_string=build_search_string(partition_out)
            service=connect()
            job=search_export(service,search_string,partition_out)
            #print_results(job)
            write_results(job,partition_out)
            update_partition_status(partition_file_in,partition_out,'completed')
            #job.cancel()
        else:
            break

    logging.info('dispatch_searches-end')

        
def write_results(job_in,partition_in):
    logging.info('write_results-start')
    earliest=partition_in["earliest"]
    earliest=re.sub("[/:]", "-", earliest)

    output_file=config.get('export', 'directory')+'/'+partition_in["index"]+"_"+earliest+".json"
    f = open(output_file, "w")
    reader = results.JSONResultsReader(job_in)
    for result in reader:
        if isinstance(result, dict):
            print(result,file=f)
        elif isinstance(result, results.Message):
            # Diagnostic messages may be returned in the results
            print(result,file=f)
    
    f.close()
    logging.info('write_results-end')

        
def connect():
    try:
        logging.info('connect-start')
        logging.debug('SPLUNK_HOST: %s',config.get('splunk', 'SPLUNK_HOST'))
        logging.debug('SPLUNK_PORT: %s',config.get('splunk', 'SPLUNK_PORT'))
        logging.debug('SPLUNK_AUTH_TOKEN: %s',config.get('splunk', 'SPLUNK_AUTH_TOKEN'))
       
        service = client.connect(
            host=config.get('splunk', 'SPLUNK_HOST'),
            port=config.get('splunk', 'SPLUNK_PORT'),
        #    username=USERNAME,
        #    password=PASSWORD)
            splunkToken=config.get('splunk', 'SPLUNK_AUTH_TOKEN'),
            autologin=True)
        logging.debug(service)
        logging.info('connect-successful')
        logging.info('connect-end')
        return service
    except:
        logging.error('connect-failed')

def main():
    partition_file='partitions.info'
    load_config()
    service=connect()
    logging.debug(service)
    date_array=explode_date_range(config.get('search', 'begin_date'),config.get('search', 'end_date'),
                              config.get('export', 'parition_units'),int(config.get('export', 'partition_interval')))
    write_search_partitions(date_array)
    create_output_dir()
    #get_search_partition(partition_file)
    dispatch_searches(partition_file)
    #job.cancel()
    #print_results2(job)
    #write_results(job)
    


if __name__ == "__main__":
    main()

09-29 18:12:07 root         INFO     connect-start
09-29 18:12:07 root         INFO     connect-successful
09-29 18:12:07 root         INFO     connect-end
09-29 18:12:07 root         INFO     explode_date_range-start
09-29 18:12:07 root         INFO     explode_date_range-end
09-29 18:12:07 root         INFO     write_search_partitions-start
09-29 18:12:07 root         INFO     write_search_partitions-end
09-29 18:12:07 root         INFO     dispatch_searches-start
09-29 18:12:07 root         INFO     build_search_string-start
09-29 18:12:07 root         INFO     build_search_string-end
09-29 18:12:07 root         INFO     connect-start
09-29 18:12:07 root         INFO     connect-successful
09-29 18:12:07 root         INFO     connect-end
09-29 18:12:07 root         INFO     search_export-start
09-29 18:12:07 root         INFO     search_export-end
09-29 18:12:07 root         INFO     write_results-start
09-29 18:12:08 root         INFO     write_results-end
09-29 18:12:08 root      

In [ ]:
with open(partition_file,'r+') as f:
    portalocker.lock(f, portalocker.LOCK_EX)
    data = json.load(f)
    search_partition_out=''
    for search_partition in data:
        if search_partition["status"]=='not started':
            pprint(search_partition["earliest"])
            search_partition["status"]='assigned'
            search_partition["pid"]=os.getpid()
            search_partition_out=search_partition
            #return search_partition
            break
            #pid = os.getpid()
    #with open("sample.json", "w") as outfile:
    #outfile.write(json_object)
    json_object = json.dumps(data, indent=4)
    f.seek(0)
    f.write(json_object)
    f.truncate()
    #f.write(json_object)
    pprint(data)

In [ ]:
partition_file='partitions.info'
f = open(partition_file)
  
# returns JSON object as 
# a dictionary
data = json.load(f)
pprint(data)

In [ ]:
def build_search_string():
    logging.info('build_search_string-start')
    logging.debug('indexes: %s',config.get('search', 'indexes'))
    
    index_list=config.get('search', 'indexes').split()
    
    logging.debug('extra: %s',config.get('search', 'extra'))
    s='search index='+config.get('search', 'indexes')
    s+=' '+config.get('search', 'extra')
    
    logging.debug('s: %s',s)
    logging.info('build_search_string-end')
    return s

search_string=build_search_string()

In [ ]:
partition_file='partitions.info'

def search_export(service_in,search_in,partition_in):
    logging.info('search_export-start')
    

    logging.debug(service_in)
    kwargs_export = {"search_mode": "normal",
                     'earliest_time': partition_in["earliest"],
                     'latest_time': partition_in["latest"],
                     "output_mode": "json"}
    job = service_in.jobs.export(search_in, **kwargs_export)

    logging.info("search_export-end")
    return job


def update_partition_status(partition_file_in,partition_in,status_in):
    with open(partition_file_in,'r+') as f:
        portalocker.lock(f, portalocker.LOCK_EX)
        data = json.load(f)
        search_partition_out=None
        for search_partition in data:
            if search_partition["id"]==partition_in["id"]:
                #pprint(search_partition["earliest"])
                search_partition["status"]=status_in
                #return search_partition
                break
            
        json_object = json.dumps(data, indent=4)
        f.seek(0)
        f.write(json_object)
        f.truncate()

        

def build_search_string(partition_in):
    logging.info('build_search_string-start')
    logging.debug('indexes: %s',config.get('search', 'indexes'))
    logging.debug('extra: %s',config.get('search', 'extra'))
    s='search index='+partition_in["index"]
    #s+=' earliest='+partition_in["earliest"]
    #s+=' latest='+partition_in["latest"]
    s+=' '+config.get('search', 'extra')
    
    logging.info('s: %s',s)
    logging.info('build_search_string-end')
    return s


def write_results(job_in,partition_in):
    logging.info('write_results-start')
    earliest=partition_in["earliest"]
    earliest=re.sub("[/:]", "-", earliest)

    output_file=config.get('export', 'directory')+'/'+partition_in["index"]+"_"+earliest+".json"
    f = open(output_file, "w")
    reader = results.JSONResultsReader(job_in)
    for result in reader:
        if isinstance(result, dict):
            print(result,file=f)
        elif isinstance(result, results.Message):
            # Diagnostic messages may be returned in the results
            print(result,file=f)
    
    #for result in results.JSONResultsReader(job_in.results(output_mode='json')):
        #print(result)
     #   print(result, file=f)
     #   f.write(result.value)

#    f.write(job_in)
    f.close()
    logging.info('write_results-end')

def dispatch_searches(partition_file_in):
    logging.info('dispatch_searches-start')
    #search_partition=get_search_partition(partition_file)
#    logging.info('
    while True:
        partition_out=get_search_partition('partitions.info')

        if partition_out is not None:
            print("Search it")
            print(partition_out["earliest"])
            search_string=build_search_string(partition_out)
            service=connect()
            job=search_export(service,search_string,partition_out)
            #print_results(job)
            write_results(job,partition_out)
            update_partition_status(partition_file_in,partition_out,'completed')
            #job.cancel()
        else:
            print("Done")
            break

    
    logging.info('dispatch_searches-end')
    
def create_output_dir():
    path=config.get('export', 'directory')
    if not os.path.exists(path):
        os.makedirs(path)
    

load_config()
service=connect()
logging.debug(service)
date_array=explode_date_range(config.get('search', 'begin_date'),config.get('search', 'end_date'),
                          config.get('export', 'parition_units'),int(config.get('export', 'partition_interval')))
write_search_partitions(date_array)
create_output_dir()
#get_search_partition(partition_file)
dispatch_searches(partition_file)

#job=search(service,search_string)
#print_results(job)
#job.cancel()

In [ ]:
def get_search_partition(partition_file_in):
    with open(partition_file_in,'r+') as f:
        portalocker.lock(f, portalocker.LOCK_EX)
        data = json.load(f)
        search_partition_out=None
        for search_partition in data:
            if search_partition["status"]=='not started':
                #pprint(search_partition["earliest"])
                search_partition["status"]='assigned'
                search_partition["pid"]=os.getpid()
                search_partition_out=search_partition
                #return search_partition
                break
                #pid = os.getpid()
        #with open("sample.json", "w") as outfile:
        #outfile.write(json_object)
        json_object = json.dumps(data, indent=4)
        f.seek(0)
        f.write(json_object)
        f.truncate()
        return search_partition_out
        #f.write(json_object)
        #pprint(data)
        
partition_out=get_search_partition('partitions.info')

if partition_out is not None:
    print("Search it")
else:
    print("Done")


In [ ]:
import re  

str = "06/26/2022:00:00:00"
print(re.sub("[/:]", "-", str))

In [ ]:
date_array=explode_date_range(config.get('search', 'begin_date'),config.get('search', 'end_date'),
                          config.get('export', 'parition_units'),int(config.get('export', 'partition_interval')))

In [ ]:
print(config.get('search', 'begin_date'))